# Progetto Python


In [1]:
import pandas as pd

In [ ]:
import os

In [ ]:
os.getcwd()

In [2]:
#carichiamo i dati degli annunci
df_listings = pd.read_csv('/Users/ivano/Documents/listings.csv')

In [ ]:
print(df_listings)

In [ ]:
df_listings.dtypes

In [3]:
#carichiamo i dati delle recensioni
df_reviews = pd.read_csv('/Users/ivano/Documents/reviews.csv')

In [ ]:
print(df_reviews)

In [ ]:
print(df_reviews.dtypes)

In [5]:
# Calcola il numero di recensioni per ogni annuncio, può essere una buona proxy per il successo di un annuncio
df_nreviews = df_reviews.groupby('listing_id').size().reset_index(name='n_reviews')

In [ ]:
print(df_nreviews)

In [6]:
df_nreviews2 = df_nreviews.rename(columns={'listing_id': 'id'})

In [ ]:
print(df_nreviews2)

In [7]:
# Combina i dati degli annunci con il numero di prenotazioni
df_listings = pd.merge(df_listings, df_nreviews2, on='id', how='left')

In [8]:
def convert_to_int(value):
    try:
        result = int(value)
    except (ValueError, TypeError):
        result = 0
    return result

In [9]:
df_listings['n_reviews'] = df_listings['n_reviews'].apply(convert_to_int)

In [ ]:
df_listings['n_reviews'].dtypes

In [ ]:
print(df_listings['n_reviews'])

In [ ]:
df_listings['price'].dtypes

In [10]:
df_listings['price'] = df_listings['price'].replace({'\$': ''}, regex=True).astype(str)

In [11]:
df_listings['price'] = df_listings['price'].str.replace(',', '').astype(float)

In [ ]:
df_listings['price'].dtypes

In [ ]:
df_listings['description'] = df_listings['description'].astype(str)

In [ ]:
df_listings['len_description'] = df_listings['description'].apply(lambda x : len(str(x)))

In [12]:
df_listings[['host_location', 'id', 'latitude', 'longitude']]

,host_location,id,latitude,longitude
0,"Milan, Italy",5678774,45.452340,9.174740
1,"Milan, Italy",5625937,45.460010,9.185820
2,NaN,6139798,45.512520,9.130460
3,"Rome, Italy",7479570,45.456200,9.201310
4,"Milan, Italy",8438872,45.495470,9.207160
...,...,...,...,...
20365,"Milan, Italy",637425034363201358,45.457314,9.180191
20366,"Milan, Italy",40817739,45.458340,9.179450
20367,"Milan, Italy",592550481216316403,45.492150,9.193410
20368,NaN,10150665,45.460520,9.180800


In [13]:
#eliminiamo le colonne di cui non ci serviremo
df_listings = df_listings.drop(['listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url', 'host_url','host_name','host_thumbnail_url', 'host_picture_url', 'host_neighbourhood','neighbourhood_cleansed','neighbourhood_group_cleansed','minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated' ], axis=1)

In [ ]:
df_listings

In [14]:
import matplotlib.pyplot as plt

In [ ]:
fig,ax = plt.subplots(figsize=(12,12), ncols=1)
ax.boxplot(df_listings['price'])
plt.tight_layout()
plt.show()

In [15]:
from scipy import stats

In [ ]:
Q1 = df_listings['price'].quantile(0.25)
Q3 = df_listings['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

df_listings = df_listings[(df_listings['price'] >= lower_bound) & (df_listings['price'] <= upper_bound)]


In [ ]:
df_listings

In [ ]:
fig,ax = plt.subplots(figsize=(12,12), ncols=1)
ax.boxplot(df_listings['price'])
plt.tight_layout()
plt.show()

In [22]:
df_success = df_listings[(df_listings['n_reviews'] >= 50)&(df_listings['availability_30'] <= 10)]

In [ ]:
print(df_success)

In [23]:
df_listings = pd.merge(df_listings, df_success['id'], on='id', how='outer', suffixes=('_listings', '_success'), indicator=True)
df_listings = df_listings[df_listings['_merge'] == 'left_only']
df_listings = df_listings.drop('_merge', axis=1)

In [ ]:
meanprice_success = df_success['price'].mean()
stdprice_success = df_success['price'].std()
meanprice_listings = df_listings['price'].mean()
stdprice_listings = df_listings['price'].std()

In [ ]:
t, p = stats.ttest_ind_from_stats(mean1=meanprice_success, std1=stdprice_success, nobs1=len(df_success),
                                  mean2=meanprice_listings, std2=stdprice_listings, nobs2=len(df_listings))


In [ ]:
if p < 0.05:
    print("La differenza tra le medie dei prezzi è significativa")
else:
    print("La differenza tra le medie dei prezzi non è significativa")

In [ ]:
meanprice_success

In [ ]:
meanprice_listings

In [ ]:
stdprice_success

In [ ]:
stdprice_listings

In [ ]:
meanlendes_success = df_success['len_description'].mean()
stdlendes_success = df_success['len_description'].std()
meanlendes_listings = df_listings['len_description'].mean()
stdlendes_listings = df_listings['len_description'].std()

In [ ]:
tlen, plen = stats.ttest_ind_from_stats(mean1=meanprice_success, std1=stdprice_success, nobs1=len(df_success),
                                  mean2=meanprice_listings, std2=stdprice_listings, nobs2=len(df_listings))


In [ ]:
if plen < 0.05:
    print("La differenza tra le medie della lunghezza delle descrizioni è significativa")
else:
    print("La differenza tra le medie della lunghezza delle descrizioni non è significativa")

In [ ]:
meanlendes_success

In [ ]:
meanlendes_listings

In [ ]:
stdlendes_success

In [ ]:
stdlendes_listings

In [16]:
import pandas as pd

In [ ]:
all_words = ' '.join(df_success['description'].astype(str).tolist())

In [ ]:
all_words

In [ ]:
def word_count(df, col_name):
    # Uniamo tutte le parole in una stringa unica
    all_words = ' '.join(df[col_name].astype(str).tolist())
   # Rimuove la punteggiatura e le cifre
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)

    # Converti il testo in una lista di parole
    words = text.split()

    # Conta le ricorrenze di ogni parola
    word_counts = {}
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    return word_counts

In [ ]:
word_success_description = word_count(df_success, description)

GEOLOCALIZZAZIONE

In [ ]:
pip install pandas-geojson

In [ ]:
pip install geopandas

In [17]:
from pandas_geojson import read_geojson
geo_json = read_geojson('/Users/ivano/Documents/neighbourhoods.geojson')

In [18]:
df_neigh = pd.read_csv('/Users/ivano/Documents/neighbourhoods.csv')

In [19]:
import geopandas as gps

In [20]:
milan_gps = gps.read_file('/Users/ivano/Documents/neighbourhoods.geojson')

In [ ]:
milan_gps.info()

In [ ]:
milan_gps.plot()

In [ ]:
milan_gps['geometry']

In [ ]:
milan_gps.centroid

In [24]:
geo_success = df_success[['id', 'latitude', 'longitude']]

In [ ]:
geo_success

In [ ]:
plt.scatter(geo_success['longitude'],geo_success['latitude'], s = 1, c = 'red')

In [25]:
df_listings[['id', 'latitude', 'longitude']]

,id,latitude,longitude
0,5678774,45.452340,9.174740
2,6139798,45.512520,9.130460
3,7479570,45.456200,9.201310
4,8438872,45.495470,9.207160
5,10609449,45.448450,9.181160
...,...,...,...
20364,4845413,45.476990,9.182210
20365,637425034363201358,45.457314,9.180191
20366,40817739,45.458340,9.179450
20367,592550481216316403,45.492150,9.193410


In [26]:
geo_unsuccess = df_listings[['id', 'latitude', 'longitude']]

In [ ]:
geo_success

In [ ]:
geo_unsuccess

In [ ]:
geo_unsuccess.info()

In [ ]:
plt.scatter(geo_unsuccess['longitude'],geo_unsuccess['latitude'], s = 1, c = 'green')

In [ ]:
plt.scatter(geo_unsuccess['longitude'],geo_unsuccess['latitude'], s = 1, c = 'green')
plt.scatter(geo_success['longitude'],geo_success['latitude'], s = 1, c = 'red')
milan_gps.plot()

In [ ]:
plt.scatter(geo_unsuccess['longitude'],geo_unsuccess['latitude'], s = 1, c = 'green')
plt.scatter(geo_success['longitude'],geo_success['latitude'], s = 1, c = 'red')

In [27]:
df_metro = pd.read_json(r'C:/Users/ivano/documenti/metromilano.json')

In [ ]:
df_metro.head()

In [28]:
df_metro.rename(columns={'DO_Y':'latitude', 'DO_X':'longitude'}, inplace=True)

In [29]:
geo_metro = df_metro.drop(['ID', 'N_ARCHI'], axis=1)

In [50]:
geo_metro

,FERMATA,latitude,longitude
0,GESSATE,45.546128,9.437564
1,ZARA,45.492662,9.192703
2,S.DONATO,45.430214,9.256324
3,MACIACHINI,45.497771,9.184904
4,MOLINO DORINO,45.505079,9.093327
...,...,...,...
88,COMASINA,45.531556,9.161898
89,AFFORI NORD,45.522379,9.167983
90,AFFORI CENTRO,45.512880,9.174410
91,DERGANO,45.505225,9.179710


In [ ]:
plt.scatter(geo_unsuccess['longitude'],geo_unsuccess['latitude'], s = 1, c = 'green')
plt.scatter(geo_success['longitude'],geo_success['latitude'], s = 1, c = 'red')
plt.scatter(geo_metro['longitude'],geo_metro['latitude'], s = 1, c = 'yellow')

In [30]:
import numpy as np

In [31]:
from numpy import sin, cos, arccos, pi, round

In [32]:
##funzioni in generale per trovare distanza date 2 coordinate terrestri

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def distanza(latitude1, longitude1, latitude2, longitude2):
    
    theta = longitude1 - longitude2
    
    distance = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) + 
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )
    return round(distance * 1609.344, 2) ##distanza in metri con due decimali 

In [33]:
distanza(geo_success['latitude'][1], geo_success['longitude'][1],geo_metro['latitude'][1], geo_metro['longitude'][1])

3670.06

In [34]:
geo_metro.loc[6]['latitude']

45.5244791938931

In [48]:
i = 17

In [49]:
distanza(geo_success.loc[i]['latitude'], geo_success.loc[i]['longitude'],
         geo_metro.loc[i]['latitude'], geo_metro.loc[i]['longitude'])

20972.43

In [43]:
geo_success

,id,latitude,longitude
0,5625937,45.46001,9.18582
1,206450,45.45238,9.20745
2,26374075,45.49226,9.22463
3,4123599,45.46024,9.18091
4,2759420,45.48264,9.18543
...,...,...,...
2160,3816798,45.47167,9.23089
2161,17899097,45.46499,9.22995
2162,3725671,45.47641,9.18230
2163,45512683,45.48123,9.17672


In [ ]:
geo_metro.loc[5]

geo_success ha gli indici sfalsati, per quello da errore per alcuni indici,
soluzioni: - si ridanno tutti gli indici (magari con un for/range), si cambia la funzione in modo che se non trovo un indice 
passa a quello successivo


sistemato questo, doppio ciclo for (1 per geo_success, 1 per geo_metro) per valutare tutte le distanze e mettere i risultati
in un altro dataframe (????)

In [39]:
geo_success.reset_index(inplace=True)
geo_success.drop(['index'], axis=1, inplace=True)

In [57]:
for i in range (2165):
    for h in range (93):
        dis = distanza(geo_success.loc[i]['latitude'], geo_success.loc[i]['longitude'],
         geo_metro.loc[h]['latitude'], geo_metro.loc[h]['longitude'])
        
        print(dis)

21830.35
3670.06
6420.77
4199.19
8780.86
5710.81
7895.48
9953.48
2918.32
3348.03
10200.21
11578.38
8461.81
7601.7
3570.29
20344.38
18908.41
17435.8
16560.92
15438.7
14194.77
13224.3
10815.33
9894.8
9632.78
10516.52
8156.89
6959.75
5993.63
5276.53
4759.39
3798.61
3381.47
3020.44
2263.37
1761.45
1242.05
568.32
226.72
886.31
3214.86
3580.9
4248.87
5032.16
2353.14
1540.48
3526.16
2850.58
2709.57
1970.92
1334.51
1144.48
1742.98
2233.37
2521.44
2942.96
3383.24
3767.58
4389.28
4992.03
8086.95
7336.94
6564.39
5572.61
4867.49
3961.66
3174.97
2681.82
3377.84
6250.02
7220.23
2295.13
1458.66
1246.99
995.11
619.15
981.78
1161.32
2974.6
2321.88
1716.06
4281.67
4707.24
5263.44
5955.44
6561.95
7158.86
8985.32
8170.77
7072.75
5945.54
5049.91
9515.43
20743.15
4624.23
4540.14
5344.33
10653.51
7382.39
8179.49
10211.29
3567.38
3160.79
12036.54
11137.04
9779.86
7803.1
3637.55
19271.48
17852.01
16359.56
15494.38
14388.14
13207.83
12317.98
10096.11
9266.27
9185.41
10074.66
7746.08
6646.6
5713.43
4880.53
4208.

2754.07
2644.73
2394.43
1109.1
1426.65
1882.0
1343.65
1609.75
2057.35
2685.33
3264.22
3845.94
5655.94
5761.21
4648.26
3498.47
2577.89
9333.9
22915.09
4787.62
6634.79
5228.94
8844.92
4996.01
9097.35
11152.67
1707.47
2390.83
10389.71
12786.21
9309.47
6402.49
4789.4
21433.06
20001.53
18521.0
17648.52
16530.31
15304.47
14354.83
11983.01
11076.22
10837.32
11722.99
9362.09
8170.52
7203.93
6471.42
5926.96
4979.78
4602.14
4228.43
3465.33
2966.06
2453.77
1778.1
1424.02
1594.81
3418.28
3767.74
4410.01
5181.56
2638.87
1994.14
4709.28
4016.27
3811.0
3020.97
2339.0
1901.93
2006.39
2336.35
2497.72
2740.82
3047.47
3283.31
3793.38
4330.55
8154.16
7434.65
6688.45
5682.84
5030.23
4095.6
3279.54
2756.75
2351.31
5047.87
6020.27
1075.55
557.77
982.72
1192.79
1771.88
1971.71
2360.44
4192.63
3537.74
2928.33
5503.09
5928.08
6481.91
7169.79
7772.36
8365.36
10181.02
9019.59
7954.27
6871.58
6022.79
9574.92
18357.64
6772.18
1900.46
7600.02
13807.79
10706.34
8812.86
10601.21
6519.16
5553.03
15072.67
10209.72
11924

8215.13
9234.87
4817.85
11189.73
2513.96
18921.37
17484.05
16101.52
15234.49
14119.66
12788.99
11693.95
9065.13
8050.23
7463.86
8253.76
6108.55
4947.87
4182.23
3976.62
4020.55
3366.34
2096.58
1699.47
1953.48
2319.22
2763.4
3313.16
3743.61
4327.58
6300.45
6614.35
7203.5
7893.43
5556.42
4862.48
1234.87
1297.1
1156.64
1872.62
2547.6
3012.36
3369.99
3545.86
3793.1
4240.22
4693.41
5226.27
5882.19
6479.93
6724.71
6006.49
5315.75
4654.23
4059.82
3726.18
3586.43
3596.63
7193.28
9838.33
10808.27
5826.29
4786.34
4189.85
3800.03
3217.48
2917.97
3235.29
2490.37
2626.67
2877.68
2525.05
2591.37
2790.41
3173.08
3593.69
4057.65
5649.29
4532.87
3407.72
2239.09
1293.67
7994.08
21252.23
1730.07
10100.99
958.81
6231.24
6798.45
5117.26
6899.93
6956.71
7789.07
7045.88
9920.41
3933.55
11493.66
3697.28
19772.25
18342.02
16988.18
16128.57
15023.55
13685.31
12573.31
9929.69
8911.7
8248.53
8988.12
6972.11
5870.84
5192.44
5083.11
5180.38
4550.48
3281.3
2869.62
3004.89
3277.74
3638.6
4073.2
4485.23
5159.32
7282.69

4728.76
5144.16
5664.46
4105.84
3799.61
1847.46
2114.07
2611.63
3019.78
3377.68
3921.65
4710.79
5145.16
5481.19
6000.65
6510.39
7017.91
7705.84
8341.46
9711.42
8970.55
8239.24
7451.13
6774.8
6172.43
5697.59
5439.4
6827.84
9943.72
10894.58
6128.18
5296.52
4927.09
4531.35
3445.28
3555.78
2696.09
966.27
1554.89
2141.06
922.45
1337.25
1902.62
2613.15
3235.01
3846.04
5703.8
7137.67
6066.73
4975.93
4120.29
11026.65
17992.07
2290.44
7060.38
3025.82
10157.32
9255.96
3995.87
6012.07
7136.06
7219.96
11025.48
7343.81
6895.33
11754.94
683.13
16500.13
15057.04
13627.02
12751.37
11625.74
10322.91
9277.45
6737.51
5766.17
5411.49
6287.12
3938.96
2725.51
1768.33
1224.75
1203.94
937.77
1001.52
1554.66
2207.92
2624.55
3069.44
3717.54
4042.91
4152.62
5139.41
5342.04
5759.87
6276.2
4691.59
4342.68
1637.01
2096.24
2597.92
3146.5
3602.8
4169.3
4916.65
5317.06
5645.21
6162.24
6669.94
7190.47
7880.5
8514.71
9542.65
8814.76
8103.71
7366.64
6714.31
6182.75
5778.99
5571.44
7337.94
10420.21
11377.98
6542.39
5662.5

5622.57
6286.39
4094.0
3495.33
701.85
436.47
922.82
1427.33
1950.61
2526.96
3223.62
3602.54
3926.67
4441.82
4948.18
5471.75
6161.53
6794.65
8057.27
7310.02
6569.48
5765.02
5085.67
4491.78
4055.76
3845.64
6201.14
9121.81
10092.95
5130.43
4166.14
3676.71
3258.77
2308.3
2233.4
1923.26
874.42
1007.21
1417.02
1596.43
1941.78
2443.71
3104.19
3697.87
4288.5
6111.3
6124.01
5003.45
3841.12
2902.72
9394.42
19261.21
757.4
8112.98
1480.45
8614.29
8116.53
4075.51
6123.38
6740.95
7135.21
9476.81
8257.52
5626.37
11437.05
1333.82
17771.55
16330.83
14929.5
14058.44
12938.44
11615.71
10536.03
7929.4
6923.53
6411.74
7240.17
4999.7
3801.0
2966.97
2697.99
2749.42
2179.52
985.16
904.98
1596.56
2095.57
2617.88
3282.08
3693.53
4085.07
5699.0
5973.24
6501.84
7129.35
5059.74
4489.74
325.48
1033.67
1403.79
2169.03
2799.31
3364.0
3959.46
4266.12
4565.17
5060.75
5551.25
6085.92
6767.37
7389.29
8006.43
7285.21
6586.59
5890.21
5265.23
4825.23
4539.67
4430.41
7197.5
10082.14
11054.78
6071.74
5079.85
4552.57
4135.21
3

6446.23
5701.52
5237.38
4184.62
7378.52
8272.21
4170.35
3791.88
3803.53
3595.62
2763.69
3192.1
2052.3
2822.78
2425.49
2180.64
3966.95
4421.09
5006.48
5726.31
6351.34
6963.55
8820.87
9545.87
8422.07
7254.6
6309.45
11943.47
18016.72
2298.27
7007.51
3042.0
10164.17
9228.88
4054.81
6070.46
7086.48
7164.93
11040.83
7386.94
6935.83
11702.56
641.18
16524.77
15081.67
13649.95
12774.1
11648.25
10347.03
9304.02
6768.94
5800.04
5452.65
6329.43
3979.26
2767.61
1808.02
1241.85
1185.7
881.91
976.7
1533.53
2175.1
2586.13
3026.8
3672.58
3995.24
4099.26
5080.37
5283.18
5701.73
6219.37
4633.01
4286.12
1638.68
2080.44
2583.22
3120.17
3568.74
4133.88
4885.2
5288.73
5617.72
6135.17
6643.24
7162.78
7852.8
8487.26
9547.58
8818.26
8105.03
7362.75
6707.89
6169.39
5758.9
5546.72
7283.68
10368.44
11325.76
6494.81
5618.4
5205.63
4798.26
3741.42
3791.79
3065.53
1228.42
1881.53
2491.83
403.71
781.13
1346.49
2061.86
2687.27
3300.94
5164.01
6685.9
5634.95
4576.32
3762.52
10832.31
21504.71
3226.18
6522.56
3778.46
8676

2638.12
2984.51
3368.05
3683.78
4254.56
4826.92
8297.99
7559.66
6796.24
5793.22
5107.46
4180.0
3368.27
2850.38
2780.27
5658.12
6626.83
1749.67
1082.96
1149.11
1106.67
1213.59
1520.83
1689.61
3531.35
2874.51
2263.11
4852.8
5285.09
5847.5
6544.72
7154.38
7753.57
9584.06
8701.57
7612.9
6498.16
5615.19
9726.33
22567.65
4475.96
6491.12
4947.41
8867.55
5236.01
8753.62
10810.48
2054.82
2611.46
10377.56
12422.11
9093.06
6743.11
4427.6
21084.8
19652.35
18173.21
17300.21
16181.16
14951.96
13998.74
11621.49
10713.21
10473.15
11358.85
8997.93
7806.67
6840.05
6107.36
5564.83
4616.33
4245.34
3880.69
3120.17
2619.27
2103.14
1427.09
1061.65
1271.17
3255.9
3614.11
4269.48
5049.52
2441.15
1736.75
4373.14
3686.28
3502.63
2724.75
2048.34
1661.86
1906.21
2296.41
2501.08
2804.96
3158.37
3443.21
3991.99
4552.08
8174.35
7443.9
6687.48
5681.81
5009.91
4076.6
3259.49
2736.16
2602.51
5390.06
6361.28
1438.81
776.77
969.85
1046.54
1436.19
1677.63
1996.45
3830.45
3175.05
2565.16
5143.81
5570.95
6127.53
6818.63
7423

7970.75
6862.4
5720.11
4807.21
9603.8
23405.1
4908.11
7210.19
5264.72
8373.88
4405.46
9310.45
11355.31
1613.78
2631.99
9953.21
13151.17
9181.42
6254.93
5127.26
21920.69
20486.42
19010.33
18136.38
17015.69
15778.48
14814.05
12408.91
11486.23
11210.44
12091.54
9734.61
8530.91
7566.57
6864.34
6353.15
5391.71
4900.77
4481.48
3715.67
3231.14
2746.29
2093.48
1817.43
2136.12
4002.68
4349.34
4986.47
5753.74
3229.3
2577.28
4910.53
4202.61
3933.77
3126.57
2452.67
1930.08
1779.56
1999.98
2096.18
2261.07
2518.47
2717.37
3208.73
3740.61
7686.9
6980.54
6247.8
5246.48
4620.47
3692.51
2889.45
2379.12
2533.11
4901.44
5872.9
895.88
155.33
764.92
1138.86
2032.92
2111.24
2754.02
4538.17
3894.4
3297.91
5814.53
6221.99
6756.25
7423.82
8011.53
8591.84
10378.05
8893.78
7855.93
6811.69
6004.86
9100.03
20402.42
3669.45
5223.03
4403.86
10039.97
7262.78
7281.23
9325.31
3996.86
3887.96
11347.99
10463.98
8847.18
8476.42
2737.74
18921.16
17490.73
16008.73
15136.78
14019.64
12800.49
11864.16
9540.77
8665.44
8507.34
9

KeyboardInterrupt: 